In [51]:
import pandas as pd
import numpy as np
import os
cwd = os.getcwd()
cwd

'/data22/bde/Git/forks/Informer2020'

In [52]:
#https://en.wikipedia.org/wiki/Non-return-to-zero
def NRZ_maker(values):
    NRZ = []
    state = None
    prior = values[0]

    for bit in values:
        if bit == prior:
            state = 1.0
        else:
            state = 0.0
        NRZ.append(state)
    return NRZ

In [53]:
test_array = np.array([6, 6, 6, 12, 12, 12, 6, 12, 12, 12, 6, 12, 12, 12, 6, 12, 12, 12, 6, 12, 12, 12, 6, 12, 12, 12, 6, 12, 12])
test_df = pd.DataFrame({'values': test_array})
values = test_df['values'].tolist()

NRZ = []
state = 0
prior = values[0]
for bit in values:
    if bit == prior:
        state = 1
    else:
        state = 0
    NRZ.append(state)
#NRZ

test_df['NRZ'] = NRZ
test_df.head()

,values,NRZ
0,6,1
1,6,1
2,6,1
3,12,0
4,12,0


In [54]:
values = test_df['values'].tolist()
test_df['NRZ2'] = NRZ_maker(values)
test_df.head()

,values,NRZ,NRZ2
0,6,1,1.0
1,6,1,1.0
2,6,1,1.0
3,12,0,0.0
4,12,0,0.0


In [55]:
PATH = '/data22/bde'
netflix_StrangerThings = f'{PATH}/Data/5Gdataset-master/5G-production-dataset/Netflix/Driving/Season3-StrangerThings/'
netflix_animated_RickandMorty = f'{PATH}/Data/5Gdataset-master/5G-production-dataset/Netflix/Driving/animated-RickandMorty/'
amazon_TheExpanse = f'{PATH}/Data/5Gdataset-master/5G-production-dataset/Amazon_Prime/Driving/Season3-TheExpanse/'
amazon_animated_AdventureTime = f'{PATH}/Data/5Gdataset-master/5G-production-dataset/Amazon_Prime/Driving/animated-AdventureTime/'
download = f'{PATH}/Data/5Gdataset-master/5G-production-dataset/Download/Driving/'
driving_streaming = f'{PATH}/Data/5Gdataset-master/5G-production-dataset/Driving-Streaming/'
driving = f'{PATH}/Data/5Gdataset-master/5G-production-dataset/Driving/'
df_list = []
file_dir = download

for file in os.listdir(file_dir)[:]:
    print(file)
    df = pd.read_csv(file_dir+file)
    #df.info()

    values = df['CellID'].tolist()
    df['NRZ'] = NRZ_maker(values)
    df['NRZ'] = df['NRZ'].astype('int32')

    nrz_one_hot = pd.get_dummies(df['NRZ'], prefix='NRZ', dtype=int)
    #nrz_one_hot.info()
    df['NRZ_0'] = nrz_one_hot['NRZ_0']
    df['NRZ_1'] = nrz_one_hot['NRZ_1']

    df['CellIDChange'] = (df['CellID'] != df['CellID'].shift()).astype('int32')

    cellid_one_hot = pd.get_dummies(df['CellIDChange'], prefix='CellIDChange', dtype=int)
    #cellid_one_hot.info()
    df['CellIDChange_0'] = cellid_one_hot['CellIDChange_0']
    df['CellIDChange_1'] = cellid_one_hot['CellIDChange_1']
    
    df = df[['Timestamp', 'CellID', 'NRZ', 'NRZ_0', 'NRZ_1', 'CellIDChange', 'CellIDChange_0', 'CellIDChange_1', 'RSRP', 'RSRQ', 'SNR', 'CQI', 'RSSI', 'NRxRSRP', 'NRxRSRQ', 'DL_bitrate', 'UL_bitrate', 'Speed']]
    df = df.replace('-', np.nan)
    df = df.replace('2147483647.0', 0)
    
    df['date'] = pd.to_datetime(df['Timestamp'], format="%Y.%m.%d_%H.%M.%S")
    df = df.drop(columns=['Timestamp'])

    #df.fillna(method='ffill', inplace=True)
    df.fillna(0, inplace=True)

    df['CellID'] = pd.to_numeric(df['CellID'])
    df['NRZ'] = pd.to_numeric(df['NRZ'])
    df['NRZ'] = pd.to_numeric(df['NRZ_0'])
    df['NRZ'] = pd.to_numeric(df['NRZ_1'])

    #df['RSRP'] = pd.to_numeric(df['RSRP'], downcast='integer')
    df['RSRP'] = pd.to_numeric(df['RSRP'])
    
    df['RSRQ'] = pd.to_numeric(df['RSRQ'])
    df['SNR'] = pd.to_numeric(df['SNR'], downcast='integer')
    df['CQI'] = pd.to_numeric(df['CQI'], downcast='integer')
    df['RSSI'] = pd.to_numeric(df['RSSI'], downcast='integer')
    df['NRxRSRP'] = pd.to_numeric(df['NRxRSRP'], downcast='integer')
    df['NRxRSRQ'] = pd.to_numeric(df['NRxRSRQ'], downcast='integer')
    df['DL_bitrate'] = pd.to_numeric(df['DL_bitrate'])
    df['UL_bitrate'] = pd.to_numeric(df['UL_bitrate'])

    #df['RSRPoffset'] = df['RSRP'] - df['NRxRSRP']
    #df['RSRQoffset'] = df['RSRQ'] - df['NRxRSRQ']

    df.drop_duplicates(subset=['date'], inplace=True)
    df.info()
    #df.info(show_counts=True)
    #print(df.head())
    #df.plot(subplots=True, layout=(3,3), figsize=(40, 25))
    #df.to_csv('5Gdataset-{}.csv'.format(file), encoding='utf-8', index=False)
    df_list.append(df)

B_2020.02.27_17.30.15.csv
<class 'pandas.core.frame.DataFrame'>
Index: 752 entries, 0 to 865
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   CellID          752 non-null    int64         
 1   NRZ             752 non-null    int64         
 2   NRZ_0           752 non-null    int64         
 3   NRZ_1           752 non-null    int64         
 4   CellIDChange    752 non-null    int32         
 5   CellIDChange_0  752 non-null    int64         
 6   CellIDChange_1  752 non-null    int64         
 7   RSRP            752 non-null    int64         
 8   RSRQ            752 non-null    int64         
 9   SNR             752 non-null    int8          
 10  CQI             752 non-null    int8          
 11  RSSI            752 non-null    int8          
 12  NRxRSRP         752 non-null    int8          
 13  NRxRSRQ         752 non-null    int8          
 14  DL_bitrate      752 non-null    int64

In [56]:
df = pd.concat(df_list, axis=0)
df.sort_values('date', inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24446 entries, 0 to 857
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   CellID          24446 non-null  int64         
 1   NRZ             24446 non-null  int64         
 2   NRZ_0           24446 non-null  int64         
 3   NRZ_1           24446 non-null  int64         
 4   CellIDChange    24446 non-null  int32         
 5   CellIDChange_0  24446 non-null  int64         
 6   CellIDChange_1  24446 non-null  int64         
 7   RSRP            24446 non-null  int64         
 8   RSRQ            24446 non-null  int64         
 9   SNR             24446 non-null  int8          
 10  CQI             24446 non-null  int8          
 11  RSSI            24446 non-null  int8          
 12  NRxRSRP         24446 non-null  int8          
 13  NRxRSRQ         24446 non-null  int16         
 14  DL_bitrate      24446 non-null  int64         
 15  UL_bitrat

In [57]:
df.head()

,CellID,NRZ,NRZ_0,NRZ_1,CellIDChange,CellIDChange_0,CellIDChange_1,RSRP,RSRQ,SNR,CQI,RSSI,NRxRSRP,NRxRSRQ,DL_bitrate,UL_bitrate,Speed,date
0,12,1,0,1,1,0,1,-99,-15,6,12,-90,-100,-15,7,1,0,2019-12-14 10:16:30
1,12,1,0,1,0,1,0,-99,-15,6,12,-90,-100,-15,7,1,1,2019-12-14 10:16:31
3,12,1,0,1,0,1,0,-102,-14,6,12,-90,-99,-15,0,0,1,2019-12-14 10:16:32
4,12,1,0,1,0,1,0,-102,-14,6,12,-90,-99,-15,9,12,1,2019-12-14 10:16:33
5,12,1,0,1,0,1,0,-102,-14,4,12,-90,-99,-14,0,0,1,2019-12-14 10:16:34


In [58]:
df = df[['date', 'CellID', 'RSRP', 'RSRQ', 'SNR', 'CQI', 'RSSI', 'NRxRSRP', 'NRxRSRQ', 'UL_bitrate', 'DL_bitrate']]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24446 entries, 0 to 857
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        24446 non-null  datetime64[ns]
 1   CellID      24446 non-null  int64         
 2   RSRP        24446 non-null  int64         
 3   RSRQ        24446 non-null  int64         
 4   SNR         24446 non-null  int8          
 5   CQI         24446 non-null  int8          
 6   RSSI        24446 non-null  int8          
 7   NRxRSRP     24446 non-null  int8          
 8   NRxRSRQ     24446 non-null  int16         
 9   UL_bitrate  24446 non-null  int64         
 10  DL_bitrate  24446 non-null  int64         
dtypes: datetime64[ns](1), int16(1), int64(5), int8(4)
memory usage: 1.4 MB


In [59]:
df.to_csv('data/5Gdataset/download.csv', encoding='utf-8', index=False)

In [60]:
# number of changes in values
(df != df.shift(axis=0)).sum(axis=0)

date          24446
CellID          699
RSRP           9295
RSRQ           7963
SNR            9224
CQI            7725
RSSI           8710
NRxRSRP        7488
NRxRSRQ        7562
UL_bitrate    22262
DL_bitrate    22631
dtype: int64